In [4]:
import numpy as np
from scipy import optimize
data = {'Fe2+/Fe3+': 3 + 0.5*np.random.rand(5), 
       'Ni+/Ni2+': 2.5 + 0.3*np.random.rand(6),
       'Mo3+/Mo6+': 5 + 0.6*np.random.rand(20)}

In [5]:
data

{'Fe2+/Fe3+': array([3.47958767, 3.20690258, 3.15767161, 3.31060199, 3.11079514]),
 'Mo3+/Mo6+': array([5.07159899, 5.50054443, 5.3803844 , 5.05655973, 5.17137889,
        5.47049034, 5.00139567, 5.36166293, 5.37077567, 5.29017114,
        5.07779824, 5.18626155, 5.0000146 , 5.58200003, 5.19557047,
        5.17529294, 5.26721923, 5.43187883, 5.28601382, 5.55787572]),
 'Ni+/Ni2+': array([2.59004953, 2.60340789, 2.65790676, 2.66378104, 2.59052402,
        2.57709107])}

In [54]:
class LLE:
    def __init__(self, data):
        self.pairs = list(data.keys())
        self.mean = {i: np.mean(data[i]) for i in self.pairs}
        self.data = data
    
    def _log_likelihood(self, sigma, means):
        _lle = 0
        n = 0
        for k, i in enumerate(self.pairs):
            _lle += np.sum(-(self.data[i] - means[i])**2 / 2. / sigma**2)
            n += len(self.data[i])
        _lle += np.log(1./sigma / np.sqrt(2*np.pi)) * n
        return _lle
    
    def direct_estimate(self):
        _variance = 0
        n = 0
        for i in self.pairs:
            _variance += np.sum((self.data[i] - self.mean[i])**2)
            n += len(self.data[i])
        return np.sqrt(_variance / n)
    
    def max_lle(self):

        inits = [0.1] + [self.mean[i] for i in self.pairs]
        def fun(x):
            sigma = x[0]
            means = {}
            for k, i in enumerate(self.pairs):
                means[i] = x[k + 1]
            return -self._log_likelihood(sigma, means)

        x = optimize.minimize(fun, inits)
        x = x.x
        return_dict = {'sigma': x[0]}
        for k, i in enumerate(self.pairs):
            return_dict.update({i: x[k + 1]})
        return return_dict
            
        

In [55]:
lle = LLE(data)

### Maximum loglikelihood estimation 

In [57]:
res = lle.max_lle()
print(res)

{'sigma': 0.1526507251479081, 'Fe2+/Fe3+': 3.2531117901014337, 'Ni+/Ni2+': 2.613793378534225, 'Mo3+/Mo6+': 5.271744372963482}


### Direct calculations

In [58]:
print('sigma = ', lle.direct_estimate())
print('Means')
print(lle.mean)

sigma =  0.15265073151925648
Means
{'Fe2+/Fe3+': 3.2531117968049785, 'Ni+/Ni2+': 2.6137933863184943, 'Mo3+/Mo6+': 5.271744380555459}
